## Kaggle Data ETL
This file will extract and transform the Kaggle data and prepare it for the sqlite database


In [1]:
import pandas as pd
import numpy as np


### Country Analysis

In [2]:
all_data = pd.read_csv('raw_data/international_matches.csv')
all_data


,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,1993-08-08,Bolivia,Uruguay,South America,South America,59,22,0,0,3,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-08-08,Brazil,Mexico,South America,North America,8,14,0,0,1,...,No,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-08-08,Ecuador,Venezuela,South America,South America,35,94,0,0,5,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-08-08,Guinea,Sierra Leone,Africa,Africa,65,86,0,0,1,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-08-08,Paraguay,Argentina,South America,South America,67,5,0,0,1,...,No,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23916,2022-06-14,Moldova,Andorra,Europe,Europe,180,153,932,1040,2,...,No,Win,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23917,2022-06-14,Liechtenstein,Latvia,Europe,Europe,192,135,895,1105,0,...,No,Lose,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN
23918,2022-06-14,Chile,Ghana,South America,Africa,28,60,1526,1387,0,...,Yes,Lose,79.0,74.0,75.5,76.7,78.2,75.5,76.0,78.2
23919,2022-06-14,Japan,Tunisia,Asia,Africa,23,35,1553,1499,0,...,No,Lose,73.0,NaN,75.2,75.0,77.5,70.8,72.3,74.0


In [3]:
# Drop unnecessary columns
tournament_data = all_data.drop(columns=['home_team_goalkeeper_score', 'away_team_goalkeeper_score', 'home_team_mean_defense_score', 'away_team_mean_defense_score',
                                'home_team_mean_offense_score', 'away_team_mean_offense_score', 'home_team_mean_midfield_score', 'away_team_mean_midfield_score'])

# Goal differential for each team
tournament_data['home_team_goal_diff'] = tournament_data.home_team_score - \
    tournament_data.away_team_score
tournament_data['away_team_goal_diff'] = tournament_data.away_team_score - \
    tournament_data.home_team_score

# Away team result
conditions = [tournament_data['home_team_result'] ==
              'Win', tournament_data['home_team_result'] == 'Lose']
choices = ['Lose', 'Win']
tournament_data['away_team_result'] = np.select(
    conditions, choices, default='Draw')

# Separate home and away data for each team and append them into a singular data frame
home_columns = ['date', 'home_team', 'home_team_continent', 'home_team_fifa_rank', 'home_team_total_fifa_points',
                'home_team_score', 'tournament', 'city', 'country', 'neutral_location', 'shoot_out', 'home_team_result', 'home_team_goal_diff']
away_columns = ['date', 'away_team', 'away_team_continent', 'away_team_fifa_rank', 'away_team_total_fifa_points',
                'away_team_score', 'tournament', 'city', 'country', 'neutral_location', 'shoot_out', 'away_team_result', 'away_team_goal_diff']

# Create a data frame from all home teams
home_data = tournament_data[home_columns]
columns = {'home_team': 'team', 'home_team_continent': 'team_continent', 'home_team_fifa_rank': 'fifa_rank',
           'home_team_total_fifa_points': 'total_fifa_points', 'home_team_score': 'goals_scored', 'home_team_result': 'result', 'home_team_goal_diff': 'goal_diff'}
home_data = home_data.rename(columns=columns)

# Create a data frame from all away teams
away_data = tournament_data[away_columns]
columns = {'away_team': 'team', 'away_team_continent': 'team_continent', 'away_team_fifa_rank': 'fifa_rank',
           'away_team_total_fifa_points': 'total_fifa_points', 'away_team_score': 'goals_scored', 'away_team_result': 'result', 'away_team_goal_diff': 'goal_diff'}
away_data = away_data.rename(columns=columns)

# Combine the data frames
combined_data = pd.concat([home_data, away_data])
combined_data = combined_data.sort_values(['team', 'date'])

# Send the combined_data to a csv named country_data
combined_data.to_csv('raw_data/country_data.csv')

combined_data


,date,team,team_continent,fifa_rank,total_fifa_points,goals_scored,tournament,city,country,neutral_location,shoot_out,result,goal_diff
6874,2003-03-16,Afghanistan,Asia,203,0,2,AFC Asian Cup qualification,Kathmandu,Nepal,True,No,Win,1
6877,2003-03-18,Afghanistan,Asia,203,0,0,AFC Asian Cup qualification,Kathmandu,Nepal,False,No,Lose,-4
7551,2003-11-19,Afghanistan,Asia,198,0,0,FIFA World Cup qualification,Ashgabat,Turkmenistan,False,No,Lose,-11
7557,2003-11-23,Afghanistan,Asia,198,0,0,FIFA World Cup qualification,Kabul,Afghanistan,False,No,Lose,-2
9263,2005-11-09,Afghanistan,Asia,199,0,0,Friendly,Dushanbe,Tajikistan,False,No,Lose,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23240,2021-11-14,Zimbabwe,Africa,118,1147,1,FIFA World Cup qualification,Harare,Zimbabwe,False,No,Draw,0
23351,2022-01-02,Zimbabwe,Africa,121,1138,0,Friendly,Yaoundé,Cameroon,True,No,Draw,0
23358,2022-01-10,Zimbabwe,Africa,121,1138,0,African Cup of Nations,Bafoussam,Cameroon,True,No,Lose,-1
23372,2022-01-14,Zimbabwe,Africa,121,1138,1,African Cup of Nations,Bafoussam,Cameroon,True,No,Lose,-1


### Match Rating Analysis

In [4]:
# Drop all NA fifa data
match_data = all_data.dropna().copy()

# Average team EA rating
match_data['home_team_mean_total_score'] = round(((match_data.home_team_goalkeeper_score + match_data.home_team_mean_defense_score +
                                                   match_data.home_team_mean_midfield_score + match_data.home_team_mean_offense_score)/4), 1)
match_data['away_team_mean_total_score'] = round(((match_data.away_team_goalkeeper_score + match_data.away_team_mean_defense_score +
                                                   match_data.away_team_mean_midfield_score + match_data.away_team_mean_offense_score)/4), 1)

# Winner of the match
conditions = [match_data.home_team_result ==
              'Win', match_data.home_team_result == 'Lose']
choices = [match_data.home_team, match_data.away_team]
match_data['winner'] = np.select(conditions, choices, default='Draw')

# Did the better EA ranked team win
match_data['team_with_better_ea_rank'] = np.where(
    match_data.home_team_mean_total_score > match_data.away_team_mean_total_score, match_data.home_team, match_data.away_team)
match_data['better_ea_team_win'] = np.where(
    match_data.winner == match_data.team_with_better_ea_rank, True, False)

# Did the better FIFA ranked team win
match_data['team_with_better_fifa_rank'] = np.where(
    match_data.home_team_fifa_rank < match_data.away_team_fifa_rank, match_data.home_team, match_data.away_team)
match_data['better_fifa_team_win'] = np.where(
    match_data.winner == match_data.team_with_better_fifa_rank, True, False)

# Send the combined_data to a csv named country_data
match_data.to_csv('raw_data/match_data.csv')

match_data


,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,home_team_mean_total_score,away_team_mean_total_score,winner,team_with_better_ea_rank,better_ea_team_win,team_with_better_fifa_rank,better_fifa_team_win
8264,2004-09-03,Spain,Scotland,Europe,Europe,3,67,0,0,1,...,80.2,79.7,81.8,89.8,81.4,Draw,Spain,False,Spain,False
8266,2004-09-04,Austria,England,Europe,Europe,90,7,0,0,2,...,90.5,88.7,91.2,76.6,89.6,Draw,England,False,England,False
8271,2004-09-04,Croatia,Hungary,Europe,Europe,25,76,0,0,3,...,71.8,75.7,70.2,78.8,72.9,Croatia,Croatia,True,Croatia,True
8276,2004-09-04,Iceland,Bulgaria,Europe,Europe,80,41,0,0,1,...,70.5,79.7,78.5,73.2,76.7,Bulgaria,Bulgaria,True,Bulgaria,True
8278,2004-09-04,Italy,Norway,Europe,Europe,9,38,0,0,2,...,79.2,81.3,79.0,92.2,79.6,Italy,Italy,True,Italy,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23909,2022-06-14,Poland,Belgium,Europe,Europe,26,2,1544,1827,0,...,80.8,85.7,85.5,80.7,85.2,Belgium,Belgium,True,Belgium,True
23911,2022-06-14,Ukraine,Republic of Ireland,Europe,Europe,27,47,1535,1449,1,...,76.5,72.7,73.8,77.1,74.5,Draw,Ukraine,False,Ukraine,False
23912,2022-06-14,Bosnia and Herzegovina,Finland,Europe,Europe,59,57,1388,1406,3,...,70.0,72.3,73.5,76.3,74.7,Bosnia and Herzegovina,Bosnia and Herzegovina,True,Finland,False
23913,2022-06-14,Romania,Montenegro,Europe,Europe,48,70,1446,1342,0,...,76.2,74.7,68.2,74.8,71.0,Montenegro,Romania,False,Romania,False
